In [1]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index.llm_predictor import HuggingFaceLLMPredictor
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from IPython.display import Markdown, display
from llama_index import ServiceContext, LangchainEmbedding
import os

In [8]:
#%pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116
#%pip install transformers datasets evaluate rouge-score py7zr
#%pip install nltk
#%pip install accelerate
#%pip install sentencepiece
#%pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 79.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 47.5 MB/s eta 0:00:0000:0100:01
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=d9462ab4f49c6fdbc5b40887482d92d8ad209e85c948f7188e71124edc7d955e
  Stored in directory: /home/jupyter/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
Note: you may need to restart the kernel to use updated packages.


In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
data_dir = "../test data/sample_articles"

In [5]:
from llama_index.prompts.prompts import SimpleInputPrompt

system_prompt = """<|SYSTEM|># StableLM Tuned (Alpha version)
- StableLM is a helpful and harmless open-source AI language model developed by StabilityAI.
- StableLM is excited to be able to help the user, but will refuse to do anything that could be considered harmful to the user.
- StableLM is more than just an information source, StableLM is also able to write poetry, short stories, and make jokes.
- StableLM will refuse to participate in anything that could harm a human.
""" 

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [6]:
import torch


hf_predictor = HuggingFaceLLMPredictor(
    max_input_size=4096, 
    max_new_tokens=2048,
    temperature=0,
    do_sample=False,
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="stabilityai/stablelm-tuned-alpha-7b",
    model_name="stabilityai/stablelm-tuned-alpha-7b",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2
Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Use pytorch device: cuda


In [10]:
service_context = ServiceContext.from_defaults(chunk_size_limit=1024, llm_predictor=hf_predictor, embed_model=embed_model)

In [11]:
documents = SimpleDirectoryReader(data_dir).load_data()
new_index = GPTVectorStoreIndex.from_documents(documents, service_context=service_context)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 16174 tokens
> [build_index_from_nodes] Total embedding token usage: 16174 tokens


In [14]:
query_engine = new_index.as_query_engine()
response = query_engine.query("Which companies made drugs to help alzheimer disease?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 11 tokens
> [retrieve] Total embedding token usage: 11 tokens


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 2151 tokens
> [get_response] Total LLM token usage: 2151 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens
> [get_response] Total embedding token usage: 0 tokens


In [15]:
print(response)

The companies that made drugs to help Alzheimer's are Eli Lilly, J.J. Abrams, and Biogen.
